# Cleaning an Analysing data

Goal: the data needs to be clean so that it ca be used for the next steps
Requirements:
- Remove “U-bahn”
- Add “n” to the end of the location if it’s missing (Gesundbrunnen, Neukoeln, Tiergarten, etc. )
- timestamp (hh:mm:ss) - change e.g. "Mitternacht" and "After Mitternacht"
- date format (yyyy-mm-dd)
- Modify the last three unrelated crime : they can be deleted if A. it's a sole incident (not combined with other crimes) AND B. the number of the crime is relatively minor



In [40]:
# load csv into df
import pandas as pd
import re
import numpy as np
import os
from datetime import datetime
from datetime import timedelta
from time import strftime
from sklearn.preprocessing import MultiLabelBinarizer

In [17]:
folder_path = '/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/csv-output'
data = []
for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        data.append(df)
concatenated_df = pd.concat(data, ignore_index=True)



In [18]:
# cur_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
# output_path = f'/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/concacted_labeled_cases_{cur_datetime}.csv'
# concatenated_df.to_csv(output_path, index=False)

In [19]:
# df_original = pd.read_csv(output_path)
# df = df_original.copy()

In [20]:
df = concatenated_df.copy()

In [21]:
df.head()

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,1
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,1
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,3
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,2
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,1


In [22]:
df.shape

(2405, 11)

In [23]:
# Number of unique_case_id before cleaning = 2303
df['unique_case_id'].nunique()

2303

## One Hot Encoding

### To be archived

In [24]:
# crime_columns = ['Homicide', 'Hate Crime - Disability', 'Hate Crime - Gender', 'Hate Crime - Gender Identity', 'Hate Crime - Religious',
#                   'Hate Crime - Sexual orientation', 'Hate Crime - Racial/Ethnicity', 'Hate Crime - Ethnicity', 'Verbal Abuse/Verbal Assault',
#                   'Property Damage', 'Drug Offenses', 'General Assault', 'Sexual Assault', 'Sexual Harassment', 'Property Crimes',
#                   'Domestic Violence', 'Missing Person', 'Traffic Incident', 'General Assault', 'Unclassified']

In [25]:
# for index, row in df.iterrows():
#     types_of_crime = str(row['type_of_crime']).split(", ")
#     for column in crime_columns:
#         df.loc[index, column] = 0
#     for crime in types_of_crime:
#         if crime in crime_columns:
#             df.loc[index, crime] = 1
# # df.to_csv('concatenated.csv', index=False)

### Try

In [26]:
df.head()

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,1
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,1
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,3
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,2
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,1


# Clean the data 

In [77]:
# Create a new DataFrame to store the cleaned data
# cleaned_df = pd.DataFrame()
cleaned_df = df.copy()

In [78]:
cleaned_df

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,1
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,1
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,3
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,2
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...
2400,000007_1327301,1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,2023,23.05,Unknow,Unknow,Male,Unknow,1
2401,000008_1327284,1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",2023,23.05,18.30 Uhr,Male and Female,Male,7,1
2402,000009_1327174,1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,2023,23.05,12.30 Uhr,Male,Male,2,1
2403,000010_1327172,1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",2023,23.05,13 Uhr,Male and Female,Unknow,40,Unknow


In [79]:
# Drop duplicates
cleaned_df.drop_duplicates(subset=['official_case_id'], keep='last', inplace=True)
cleaned_df.shape # We have 2301 unqique rows/cases

(2301, 11)

In [80]:
# pd.concat([df[[0]], df[1].str.split(', ', expand=True)], axis=1)t

In [81]:
def transform_strs_to_lst(strings):
    return [ele.strip() for ele in strings.split(',')]

In [83]:
cleaned_df['lst_type_of_crime'] = cleaned_df['type_of_crime'].apply(transform_strs_to_lst)
cleaned_df.head()

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders,lst_type_of_crime
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,1,[Property Damage]
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,1,[Homicide]
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,3,"[General Assault, Verbal Abuse, Property Damag..."
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,2,"[Property Crime, General Assault]"
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,1,[Property Crime]


In [85]:
mlb = MultiLabelBinarizer()
cleaned_df_test = cleaned_df.join(pd.DataFrame(mlb.fit_transform(cleaned_df.pop('lst_type_of_crime')),
                          columns=mlb.classes_,
                          index=cleaned_df.index))


In [86]:
# mlb = MultiLabelBinarizer(sparse_output=True)
# cleaned_df_test = cleaned_df.join(
#             pd.DataFrame.sparse.from_spmatrix(
#                 mlb.fit_transform(cleaned_df.pop('new_toc')),
#                 index=cleaned_df.index,
#                 columns=mlb.classes_))


In [87]:
cleaned_df_test

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Verstoßes gegen das Waffengesetz,Verstöße gegen das Sprengstoffgesetz,Verwahrungsbruch,Violation of Assembly Freedom Law,Violation of Infection Protection Measures,War Crimes,Widerstand gegen Vollstreckungsbeamte,Widerstand gegen und tätlichen Angriffs auf Vollstreckungsbeamte,and Manufacturing of Illegal Substances,üble Nachrede und Verleumdung
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,...,0,0,0,0,0,0,0,0,0,0
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,...,0,0,0,0,0,0,0,0,0,0
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,...,0,0,0,0,0,0,0,0,0,0
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,...,0,0,0,0,0,0,0,0,0,0
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,000007_1327301,1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,2023,23.05,Unknow,Unknow,Male,Unknow,...,0,0,0,0,0,0,0,0,0,0
2401,000008_1327284,1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",2023,23.05,18.30 Uhr,Male and Female,Male,7,...,0,0,0,0,0,0,0,0,0,0
2402,000009_1327174,1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,2023,23.05,12.30 Uhr,Male,Male,2,...,0,0,0,0,0,0,0,0,0,0
2403,000010_1327172,1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",2023,23.05,13 Uhr,Male and Female,Unknow,40,...,0,0,0,0,0,0,0,0,0,0


In [91]:
cleaned_df_test.shape

(2301, 148)

In [89]:
cleaned_df_test['War Crimes'].sum()

1

In [ ]:
pd.get_dummies(cleaned_df[])
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df['B'])
# Drop column B as it is now encoded
df = df.drop('B',axis = 1)
# Join the encoded df
df = df.join(one_hot)
df  

In [30]:
# Combine' Hate Crime - Racial/Ethnicity' and 'Hate Crime - Ethnicity'
conditions = [(cleaned_df['Hate Crime - Racial/Ethnicity'] == 0) & (cleaned_df['Hate Crime - Ethnicity'] == 0),
              (cleaned_df['Hate Crime - Racial/Ethnicity'] == 1) & (cleaned_df['Hate Crime - Ethnicity'] == 0),
              (cleaned_df['Hate Crime - Racial/Ethnicity'] == 0) & (cleaned_df['Hate Crime - Ethnicity'] == 1),
              (cleaned_df['Hate Crime - Racial/Ethnicity'] == 1) & (cleaned_df['Hate Crime - Ethnicity'] == 1)]
choices = [0, 1, 1, 1]
cleaned_df['updated_re'] = np.select(conditions, choices)

KeyError: 'Hate Crime - Racial/Ethnicity'

In [ ]:
# Quality check
cleaned_df.loc[(cleaned_df['Hate Crime - Racial/Ethnicity'] == 1) & (cleaned_df['Hate Crime - Ethnicity'] == 0)].head() 

In [ ]:
# Replace the original column
cleaned_df = cleaned_df.drop(columns=['Hate Crime - Racial/Ethnicity', 'Hate Crime - Ethnicity'])
cleaned_df['Hate Crime - Racial/Ethnicity'] = cleaned_df['updated_re']
# cleaned_df.loc[cleaned_df['updated_re'] ==0] # Quality check

In [ ]:
cleaned_df = cleaned_df.drop(columns=['updated_re'])

In [ ]:
cleaned_df.columns

In [ ]:
# Task 1: Convert 'Mitternacht' to '00:00'
cleaned_df['time'] = df['time'].replace('Mitternacht', '00.00')

In [ ]:
# Task 2: Convert 'kurz nach Mitternacht' to '00:05'
cleaned_df['time'] = df['time'].replace('kurz nach Mitternacht', '00.05')

# Task 3: Convert 'Nachmittag' to '15:00'
cleaned_df['time'] = cleaned_df['time'].replace('Nachmittag', '15.00')

# Task 3: Convert 'Nacht' to '00:00', 'Mitta' to 15.00, 'Nach' zu 00.00 and 'Vormi' zu 11.00
cleaned_df['time'] = cleaned_df['time'].replace('Nacht', '00.00')
cleaned_df['time'] = cleaned_df['time'].replace('Mitta', '15.00')
cleaned_df['time'] = cleaned_df['time'].replace('Nach', '00.00')
cleaned_df['time'] = cleaned_df['time'].replace('Vormi', '11.00')

# Task 4: Remove the second time in a timespan
cleaned_df['time'] = cleaned_df['time'].apply(lambda x: x.split('-')[0].strip() if isinstance(x, str) and '-' in x else x)

In [ ]:
# Task 5: Remove 'Uhr' from the time values
cleaned_df['time'] = cleaned_df['time'].str.replace(' Uhr', '')

In [ ]:
# Task 6 Add '.00' to times without minutes
cleaned_df['time'] = cleaned_df['time'].apply(lambda x: x + '.00' if isinstance(x, str) and re.match(r'^\d+(:\d+)?$', x) else x)

In [ ]:
# Task 7: Remove the second time in a timespan
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: x.split('-')[0].strip() if isinstance(x, str) and '-' in x else x)
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: x.split('&')[0].strip() if isinstance(x, str) and '&' in x else x)
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: x.split(',')[0].strip() if isinstance(x, str) and ',' in x else x)

In [ ]:
# Task 8: split and add 0 date
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: ('0' + x) if len(x.split('.')[0]) < 2 else (x.split('.')[0]+'.0'+x.split('.')[1]) if len(x.split('.')[1]) < 2 else ('0'+x.split('.')[0]+'.0'+x.split('.')[1]) if (len(x.split('.')[0]) < 2 and len(x.split('.')[1]) < 2) else x)

In [ ]:
# Task 9: split and add 0 tim3
cleaned_df['time'] = cleaned_df['time'].apply(lambda x: ('0' + x) if len(x.split('.')) > 1 and len(x.split('.')[0]) < 2 else (x.split('.')[0]+'.0'+x.split('.')[1]) if len(x.split('.')) > 1 and len(x.split('.')[1]) < 2 else ('0'+x.split('.')[0]+'.0'+x.split('.')[1]) if len(x.split('.')) > 1 and len(x.split('.')[0]) < 2 and len(x.split('.')[1]) < 2 else x)



In [ ]:
# Check status
cleaned_df['unique_case_id'].nunique()
cleaned_df.head(10)

## Timestamp and date

In [ ]:
cleaned_df.head(5)

In [ ]:
# Replace "." with "-" in the 'date' column
cleaned_df['date'] = cleaned_df['date'].str.replace('.', '-')

# Create datetime column
cleaned_df['datetime'] = pd.to_datetime(cleaned_df.year + '-' + 
                                        cleaned_df.date + ' ' +
                                        cleaned_df.time, 
                                        format='%Y-%d-%m %H.%M',
                                        errors='coerce' # replace any invalid dates with NaT, but be aware some have date
                                       )
# Create date column
cleaned_df['year_date'] = pd.to_datetime(cleaned_df.year + '-' + 
                                        cleaned_df.date,
                                        format='%Y-%d-%m', 
                                        errors='coerce' # replace any invalid dates with NaT, but be aware some have date
                                       )


In [ ]:
cleaned_df.head()

In [ ]:
# Get number of NA for datetime (some might have date data)
cleaned_df['datetime'].isna().sum()

# Add the missing "n"

In [ ]:
# Examine whether n is missing
print('Number of Gesundbrunne? ' + f"{cleaned_df['location'].str.contains('Gesundbrunne$', regex=True).sum()}")
print('Number of Neuköll? ' + f"{cleaned_df['location'].str.contains('Neuköll$', regex=True).sum()}")
print('Number of Tiergarte? ' + f"{cleaned_df['location'].str.contains('Tiergarte$', regex=True).sum()}")
print('Number of Berli? ' + f"{cleaned_df['location'].str.contains('Berli$', regex=True).sum()}")

In [ ]:
import re
# List of words to modify
words_to_modify = ['Gesundbrunne', 'Neuköll', 'Tiergarte', 'Berli']

# Define the regular expression pattern
pattern = r'\b(' + '|'.join(words_to_modify) + r')\b'

# Update the values in the 'location' column using regular expressions
cleaned_df['location'] = cleaned_df['location'].apply(lambda x: re.sub(pattern, r'\1n', x))

# Print the updated dataframe
cleaned_df.head(5)

In [ ]:
# Examine whether still exists
print('Number of Gesundbrunne? ' + f"{cleaned_df['location'].str.contains('Gesundbrunne$', regex=True).sum()}")
print('Number of Neuköll? ' + f"{cleaned_df['location'].str.contains('Neuköll$', regex=True).sum()}")
print('Number of Tiergarte? ' + f"{cleaned_df['location'].str.contains('Tiergarte$', regex=True).sum()}")
print('Number of Berli? ' + f"{cleaned_df['location'].str.contains('Berli$', regex=True).sum()}")

# Remove U-Bahnhof

In [ ]:
# Remove 'U-Bahnhof' from values in the 'location' column
cleaned_df['location'] = cleaned_df['location'].str.replace('U-Bahnhof', '')
cleaned_df['location'] = cleaned_df['location'].str.replace('S-Bahnhof', '')
cleaned_df['location'] = cleaned_df['location'].str.replace('U-Bahn', '')
cleaned_df['location'] = cleaned_df['location'].str.replace('S-Bahn', '')

In [ ]:
# Print the updated dataframe
cleaned_df.head(20)

# Save Cleaned data in new CSV

In [ ]:
# Copy Dataframe
final_df = cleaned_df.copy()

In [ ]:
final_df.head(5)

In [ ]:
# drop not needed columns: unique_case_id
final_df = final_df.drop(columns=['unique_case_id', 'year', 'date'])
final_df.head()

In [ ]:
final_df.columns = map(str.lower, final_df.columns)
final_df.columns = [col.replace(' ', '_').replace('_-_', '-') for col in final_df.columns]

In [ ]:
final_df.columns

In [ ]:
arranged_col_lst = [
    'official_case_id', 'type_of_crime', 'location', 
    'datetime', 'year_date',
    'time', 
    'victim_sex', 'offender_sex', 'number_of_victims', 'number_of_offenders', 'homicide',
   'hate_crime-disability', 'hate_crime-gender',
   'hate_crime-gender_identity', 'hate_crime-religious',
   'hate_crime-sexual_orientation', 'hate_crime-racial/ethnicity',
    'verbal_abuse/verbal_assault',
   'property_damage', 'drug_offenses', 'general_assault', 'sexual_assault',
   'sexual_harassment', 'property_crimes', 
#         'domestic_violence','missing_person', 'traffic_incident', 
        'unclassified'
       
                    ]

final_df = final_df[arranged_col_lst] 

In [ ]:
final_df['verbal_abuse/verbal_assault'].sum()

In [ ]:
final_df

In [ ]:
cur_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
final_df.to_csv(f"/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/final-crime-data_{cur_datetime}.csv")


## Remove Berlin-wide cases

In [ ]:
wip_df = final_df.copy()

In [ ]:
# How many Berlinweit do we have
wip_df.loc[wip_df['location'] == 'Berlinweit'].shape[0] 

In [ ]:
# How many Berlin do we have
wip_df.loc[wip_df['location'] == 'Berlin'].shape[0]

In [ ]:
wip_df = wip_df.loc[wip_df['location'] != 'Berlinweit']

In [ ]:
# How many Berlinweit do we have
wip_df.loc[wip_df['location'] == 'Berlinweit'].shape[0] 

In [ ]:
# How many Berlin do we have
wip_df = wip_df.loc[wip_df['location'] != 'Berlin']

In [ ]:
# How many Berlinweit do we have
wip_df.loc[wip_df['location'] == 'Berlinweit'].shape[0] 

In [ ]:
wip_df.shape

## Adding timeslot

In [ ]:
# Sort df
wip_df = wip_df.sort_values(by=['datetime'])

# Remove data outside 2021/9/13 and 2022/5/24
start_time = datetime(2021, 9, 13, 0, 0)  # Starting datetime
end_time = datetime(2023, 5, 25, 0, 0)  # Ending datetime
wip_df = wip_df.loc[(wip_df['datetime'] >= start_time) & (wip_df['datetime'] <= end_time)].reset_index(drop=True)
wip_df.head()

In [ ]:
def get_time_slot(tval):
    """ Return the label associated with the timestamp """
    labels = ['Midnight', 'Early Morning', 'Morning', 'Afternoon', 'Evening', 'Night']
    slot_start = [(0, 0), (4, 0), (8, 0), (12, 0), (16, 0), (20, 0)]
    for lidx, tme in enumerate(slot_start):
        if tme[0] > tval.hour:
            return labels[lidx-1]
        elif tval.hour == tme[0] and tme[1] <= tval.minute:
            return labels[lidx]
    return labels[-1]  

In [ ]:
wip_df['time_slot'] = wip_df.apply(lambda row: get_time_slot(row.datetime), axis=1)

In [ ]:
wip_df[['datetime', 'time_slot']]

In [ ]:
wip_df.shape

In [ ]:
# cur_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
# wip_df.to_csv(f"/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/final-crime-data_{cur_datetime}.csv")
